# Database Engineering

* Uses Pandas to read the cleaned measurements and stations CSV data.
* Uses the SQLAlchemy engine and connection string to create a database called hawaii.sqlite.
* Uses the declarative_base and creates ORM classes for each table.
* Creates the tables in the database using create_all.

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import os


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [2]:
# Read csv file to the df 

Hawaii_measure_df = pd.read_csv('clean_hawaii_measurements.csv', index_col=0)

Hawaii_station_df = pd.read_csv('hawaii_stations.csv')
Hawaii_station_df.head()
Hawaii_measure_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [4]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaiidb.sqlite")

In [5]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [6]:
# Use `declarative_base` from SQLAlchemy to connect your class to your sqlite database
Base = declarative_base()

In [7]:
# Creating class for Measurement table 

class Measurement(Base):
    __tablename__ = 'Measurement'
    __table_args__ = {'extend_existing': True}

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [8]:
# Creating class for station table 

class Station(Base):
    __tablename__ = 'Station'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    ELEVATION = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.station}"

In [11]:
# Use `create_all` to create the customers table in the database
Base.metadata.create_all(engine)

In [12]:
data = Hawaii_station_df.to_dict(orient='records')

In [1]:
#print(data)

In [14]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [15]:
# Save the reference to the `customers` table as a variable called `table`
Measure_table = sqlalchemy.Table('Measurement', metadata, autoload=True)

In [16]:
# Save the reference to the `customers` table as a variable called `table`
Station_table = sqlalchemy.Table('Station', metadata, autoload=True)

In [17]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(Measure_table.delete())

In [18]:
conn.execute(Station_table.delete())

In [19]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(Station_table.insert(), data)

In [20]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, None),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, None),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, None),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, None),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, None)]

In [21]:
Mdata =Hawaii_measure_df.to_dict(orient='records')

In [2]:
#print(Mdata)

In [23]:
# Use `table.insert()` to insert the data into the Measurement table
# The SQL table is populated during this step
conn.execute(Measure_table.insert(), Mdata)

In [25]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]